In [ ]:
from transformers import BertForQuestionAnswering
from transformers import AdamW
from transformers import get_linear_schedule_with_warmup
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader
from .squad_dataloader import SquadDataset
from .squad_args import squad_args
import random

args = squad_args()
device = args.device
batch_size = args.batch_size
n_epoch = args.n_epoch
learning_rate = args.learning_rate

#Load train and test dataset
train_dataset = SquadDataset('./SQuAD_train.json')
train_loader = DataLoader(train_dataset, batch_size= batch_size, shuffle=True, collate_fn=, drop_last= True)

test_dataset = SquadDataset('./SQuAD_test.json')
train_loader = DataLoader(train_dataset, batch_size= batch_size, shuffle=True, collate_fn=, drop_last= True)

#Load model
model = BertForQuestionAnswering.from_pretrained('bert-base-uncased')
model.to(device)

#Criterion, Optimizer and Scheduler
criterion = nn.CrossEntropyLoss()
optimizer = AdamW(model.parameters(), lr = learning_rate, correct_bias = False) #default lr  = 5e-5, eps = 1e-8
total_steps = len(train_loader) * n_epoch
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps = 0, num_training_steps = total_steps)

#random seed
seed_val = 100
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

#gradient init
model.zero_grad()


In [ ]:
for epoch in range(n_epoch):
    s